In [3]:
import pygad
import numpy as np
import random

# 1. Визначення класу Product та створення списку товарів.
class Product:
    def __init__(self, name, space, price):
        self.name = name
        self.space = space
        self.price = price

# Створення списку товарів
products_list = [
    Product('Refrigerator A', 0.751, 999.90),
    Product('Cell phone', 0.00000899, 2199.12),
    Product('TV 55', 0.400, 4346.99),
    Product("TV 50' ", 0.290, 3999.90),
    Product("TV 42' ", 0.200, 2999.00),
    Product("Notebook A", 0.00350, 2499.90),
    Product("Ventilator", 0.496, 199.90),
    Product("Microwave A", 0.0424, 308.66),
    Product("Microwave B", 0.0544, 429.90),
    Product("Microwave C", 0.0319, 299.29),
    Product("Refrigerator B", 0.635, 849.00),
    Product("Refrigerator C", 0.870, 1199.89),
    Product("Notebook B", 0.498, 1999.90),
    Product("Notebook C", 0.527, 3999.00)
]

# 2. Формування списків цін та просторів
prices = [product.price for product in products_list]
spaces = [product.space for product in products_list]

# Задання ліміту на використання простору (наприклад, 3 одиниці)
space_limit = 3

# 3. Визначення функції пристосованості (fitness)
def fitness_func(ga_instance, solution, solution_idx):
    """
    Функція приймає:
    1. ga_instance - екземпляр класу pygad.GA (можна не використовувати, але має бути в сигнатурі),
    2. solution - рішення (хромосома) як список або numpy-масив з 0 та 1,
    3. solution_idx - індекс рішення в популяції.
    
    Якщо ген i == 1, товар i вибрано: його вартість додається до загального балу,
    а його об'єм – до сумарного використання простору.
    Якщо використаний простір перевищує ліміт, повертається штраф (score = 1).
    """
    score = 0
    sum_spaces = 0

    # Обчислюємо score та сумарний об'єм використаного простору
    for gene, price, space in zip(solution, prices, spaces):
        if gene == 1:
            score += price
            sum_spaces += space

    # Якщо ліміт простору перевищено, повертаємо штраф
    if sum_spaces > space_limit:
        return 1

    return score

In [5]:
# 4. Визначення параметрів генетичного алгоритму
num_genes = len(products_list)
sol_per_pop = 10
num_generations = 50
num_parents_mating = 5

# 5. Експерименти з різними типами кросовера та мутацій
# Функція для генерації початкової популяції
def create_initial_population(sol_per_pop, num_genes):
    population = []
    for _ in range(sol_per_pop):
        chromosome = [1 if random.random() < 0.5 else 0 for _ in range(num_genes)]
        population.append(chromosome)
    return np.array(population)


crossover_types = ['single_point', 'two_points', 'uniform']
mutation_types = ['random', 'inversion']

for crossover in crossover_types:
    for mutation in mutation_types:
        print(f"\nЗапуск GA з кросовером: {crossover} та мутацією: {mutation}")

        initial_population = create_initial_population(sol_per_pop, num_genes)

        ga_instance = pygad.GA(
            num_generations=num_generations,
            num_parents_mating=num_parents_mating,
            fitness_func=fitness_func,
            sol_per_pop=sol_per_pop,
            num_genes=num_genes,
            initial_population=initial_population,
            gene_type=int,
            parent_selection_type="sss",       # можна змінити, якщо потрібно
            crossover_type=crossover,          # використання поточного типу кросовера
            mutation_type=mutation,            # використання поточного типу мутації
            mutation_percent_genes=10
        )

        ga_instance.run()
        solution, solution_fitness, solution_idx = ga_instance.best_solution()

        print("Найкраще рішення:", solution)
        print("Фітнес найкращого рішення:", solution_fitness)

        # Виведення вибраних товарів
        selected_products = [products_list[i].name for i in range(num_genes) if solution[i] == 1]
        print("Вибрані товари:", selected_products)


Запуск GA з кросовером: single_point та мутацією: random
Найкраще рішення: [0 1 1 1 1 1 1 0 1 1 1 0 0 1]
Фітнес найкращого рішення: 21822.0
Вибрані товари: ['Cell phone', 'TV 55', "TV 50' ", "TV 42' ", 'Notebook A', 'Ventilator', 'Microwave B', 'Microwave C', 'Refrigerator B', 'Notebook C']

Запуск GA з кросовером: single_point та мутацією: inversion
Найкраще рішення: [1 1 1 1 1 1 0 1 1 1 0 0 1 1]
Фітнес найкращого рішення: 24081.560000000005
Вибрані товари: ['Refrigerator A', 'Cell phone', 'TV 55', "TV 50' ", "TV 42' ", 'Notebook A', 'Microwave A', 'Microwave B', 'Microwave C', 'Notebook B', 'Notebook C']

Запуск GA з кросовером: two_points та мутацією: random
Найкраще рішення: [0 1 1 0 1 1 0 0 1 1 0 1 1 1]
Фітнес найкращого рішення: 19972.989999999998
Вибрані товари: ['Cell phone', 'TV 55', "TV 42' ", 'Notebook A', 'Microwave B', 'Microwave C', 'Refrigerator C', 'Notebook B', 'Notebook C']

Запуск GA з кросовером: two_points та мутацією: inversion
Найкраще рішення: [0 1 1 1 1 1 0 1 

## Висновки щодо впливу кросоверів та мутацій

Під час експериментів були опробовані різні комбінації типів кросовера та мутацій, що призвело до наступних результатів:

### Результати експериментів

- **Single-point кросовер + Random мутація:**
  - **Фітнес:** 21822.0  
  - **Вибрані товари:** Cell phone, TV 55, TV 50', TV 42', Notebook A, Ventilator, Microwave B, Microwave C, Refrigerator B, Notebook C

- **Single-point кросовер + Inversion мутація:**
  - **Фітнес:** 24081.56  
  - **Вибрані товари:** Refrigerator A, Cell phone, TV 55, TV 50', TV 42', Notebook A, Microwave A, Microwave B, Microwave C, Notebook B, Notebook C

- **Two-points кросовер + Random мутація:**
  - **Фітнес:** 19972.99  
  - **Вибрані товари:** Cell phone, TV 55, TV 42', Notebook A, Microwave B, Microwave C, Refrigerator C, Notebook B, Notebook C

- **Two-points кросовер + Inversion мутація:**
  - **Фітнес:** 24281.55  
  - **Вибрані товари:** Cell phone, TV 55, TV 50', TV 42', Notebook A, Microwave A, Microwave B, Microwave C, Refrigerator C, Notebook B, Notebook C

- **Uniform кросовер + Random мутація:**
  - **Фітнес:** 20673.71  
  - **Вибрані товари:** Cell phone, TV 55, TV 50', TV 42', Notebook A, Ventilator, Microwave B, Notebook B, Notebook C

- **Uniform кросовер + Inversion мутація:**
  - **Фітнес:** 24281.55  
  - **Вибрані товари:** Cell phone, TV 55, TV 50', TV 42', Notebook A, Microwave A, Microwave B, Microwave C, Refrigerator C, Notebook B, Notebook C

### Основні спостереження

- **Вплив мутації:**
  - **Inversion мутація** стабільно дає вищі значення фітнесу порівняно з **Random мутацією**. Для кожного типу кросовера застосування inversion мутації дозволило отримати рішення з максимальною сумою вартості товарів.
  
- **Вплив кросовера:**
  - При використанні **Inversion мутації** оптимальні результати спостерігаються як при застосуванні **Two-points**, так і **Uniform** кросовера (обидва варіанти показали фітнес ≈ 24281.55).
  - Для **Random мутації** кращий результат спостерігається при Single-point кросовері, хоча його значення фітнесу все ще значно нижче, ніж при inversion мутації.

## Висновок

Оптимальним набором параметрів для розв'язання поставленої задачі є:

- **Мутація:** Inversion
- **Кросовер:** Two-points або Uniform

Ці налаштування забезпечують найвищий фітнес (24281.55) та оптимальний набір товарів. Генетичний алгоритм продемонстрував свою ефективність у пошуку оптимальних рішень за умови правильного вибору параметрів еволюційного процесу.

---